# Importing Python Packages

In [4]:
import openpyxl
import xlrd
import os
import glob

from pandas import DataFrame as df
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean

In [5]:
# import python packages

# import nltk (natural language tool kit)
import nltk
# stopwords, FreqDist, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import requests
from lxml import html
import csv
import pandas as pd

from os import path

import matplotlib.pyplot as plt
%matplotlib inline


#regular expression
import re
#csv
import csv
# Excel-like format
import pandas as pd
#data visualization
import matplotlib.pyplot as plt
% matplotlib inline

import numpy as np

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Importing Data

## Opening the Correct Directory

In [6]:
os.chdir('C:\\Users\\millsj.ADMIN\\Desktop\\GENBA 894\\2018_DataChallenges_Teradata\\National Teams\\')

In [7]:
# filenames
excel_names = ["2013-2017 National Team Activity.xlsx"]

# read them in
excels = [pd.ExcelFile(name) for name in excel_names]

# turn them into dataframes
frames = [x.parse(x.sheet_names[0],index_col=None) for x in excels]

# delete the first row for all frames except the first
# i.e. remove the header row -- assumes it's the first
frames[1:] = [df[1:] for df in frames[1:]]

# concatenate them..
nationalteams = pd.concat(frames)

# Check headers
nationalteams.head(3)

,National Team Name,Event Chapter,Event Category,Event Type,Local Team Name,Event Name,Event Start Date,Location,Corporate Name,Revenue Raised,...,Local Team Captain Call Status,Local Team Captain Contact By Phone OK? (Y/N),Local Team Captain City,Local Team Captain State/Province,Local Team Captain Zip Code,Local Team Captain Bad Address Flag,Local Team Captain Contact By Mail OK? (Y/N),Local Team Captain Active Flag,Local Team Captain Deceased Flag,Local Team Captain Primary Connection To MS
0,Chevron Bike Team,ALC,Bike,MS 150 (2 Day Bike),Chevron,ALC 2016 Bike MS: Mississippi 150,2016-10-08,NaN,NaN,1745.0,...,G,Yes,Mobile,AL,36695.0,No,No,Yes,No,None
1,EMD Serono Bike team,ALC,Bike,Bike 2 Day,EMD Serono,ALC 2017 Bike MS: Tour De Beach,2017-09-16,Hampton Inns and Suite,NaN,390.0,...,G,Yes,Mobile,AL,36608.0,No,Yes,Yes,No,Friend has MS
2,NaN,ALC,Bike,Bike 2 Day,Freewheelin',ALC 2017 Bike MS: Tour De Beach,2017-09-16,Hampton Inns and Suite,NaN,400.0,...,G,Yes,Pensacola,FL,32506.0,No,Yes,Yes,No,None


In [8]:
# convert string to integers
#biketeams['Team Captain Accept Email'] = (biketeams['Team Captain Accept Email'] == 'TRUE').astype(int)

In [19]:
# make sure the conversion worked
#biketeams.head(3)

# Looking for Data Anomalies

In [1]:
# List of all headers
# nationalteams['National Team Name','Event Chapter','Event Category','Event Type','Local Team Name','Event Name','Event Start Date','Location','Corporate Name','Revenue Raised','Number of Team Members (local)','Local Team Captain Member ID','Local Team Captain Contact ID','Local Team Captain Email Domain','Local Team Captain Bad Email Flag','Local Team Captain Contact By Email OK? (Y/N)','Local Team Captain Phone Type','Local Team Captain Call Status','Local Team Captain Contact By Phone OK? (Y/N)','Local Team Captain City','Local Team Captain State/Province','Local Team Captain Zip Code','Local Team Captain Bad Address Flag','Local Team Captain Contact By Mail OK? (Y/N)','Local Team Captain Active Flag','Local Team Captain Deceased Flag','Local Team Captain Primary Connection To MS']

In [12]:
# counting zero values
print((nationalteams[['National Team Name','Event Chapter','Event Category','Event Type','Local Team Name','Event Name','Event Start Date','Location','Corporate Name','Revenue Raised','Number of Team Members (local)','Local Team Captain Member ID','Local Team Captain Contact ID','Local Team Captain Email Domain','Local Team Captain Bad Email Flag','Local Team Captain Contact By Email OK? (Y/N)','Local Team Captain Phone Type','Local Team Captain Call Status','Local Team Captain Contact By Phone OK? (Y/N)','Local Team Captain City','Local Team Captain State/Province','Local Team Captain Zip Code','Local Team Captain Bad Address Flag','Local Team Captain Contact By Mail OK? (Y/N)','Local Team Captain Active Flag','Local Team Captain Deceased Flag','Local Team Captain Primary Connection To MS']] == 0).sum())


National Team Name                                0
Event Chapter                                     0
Event Category                                    0
Event Type                                        0
Local Team Name                                   0
Event Name                                        0
Event Start Date                                  0
Location                                          0
Corporate Name                                    0
Revenue Raised                                   83
Number of Team Members (local)                    0
Local Team Captain Member ID                      0
Local Team Captain Contact ID                     0
Local Team Captain Email Domain                   0
Local Team Captain Bad Email Flag                 0
Local Team Captain Contact By Email OK? (Y/N)     0
Local Team Captain Phone Type                     0
Local Team Captain Call Status                    0
Local Team Captain Contact By Phone OK? (Y/N)     0
Local Team C

In [13]:
# Counting missing values
nationalteamsnull=(nationalteams[['National Team Name','Event Chapter','Event Category','Event Type','Local Team Name','Event Name','Event Start Date','Location','Corporate Name','Revenue Raised','Number of Team Members (local)','Local Team Captain Member ID','Local Team Captain Contact ID','Local Team Captain Email Domain','Local Team Captain Bad Email Flag','Local Team Captain Contact By Email OK? (Y/N)','Local Team Captain Phone Type','Local Team Captain Call Status','Local Team Captain Contact By Phone OK? (Y/N)','Local Team Captain City','Local Team Captain State/Province','Local Team Captain Zip Code','Local Team Captain Bad Address Flag','Local Team Captain Contact By Mail OK? (Y/N)','Local Team Captain Active Flag','Local Team Captain Deceased Flag','Local Team Captain Primary Connection To MS']])

print(nationalteamsnull.isnull().sum())




National Team Name                               162
Event Chapter                                      0
Event Category                                     0
Event Type                                         0
Local Team Name                                    0
Event Name                                         0
Event Start Date                                   0
Location                                         190
Corporate Name                                   810
Revenue Raised                                     0
Number of Team Members (local)                    63
Local Team Captain Member ID                      60
Local Team Captain Contact ID                     60
Local Team Captain Email Domain                   60
Local Team Captain Bad Email Flag                 60
Local Team Captain Contact By Email OK? (Y/N)     60
Local Team Captain Phone Type                     68
Local Team Captain Call Status                    68
Local Team Captain Contact By Phone OK? (Y/N) 

In [16]:
#compute count of values in columns
nationalteams['Local Team Captain Primary Connection To MS'].value_counts()

Friend has MS      316
Relative has MS    103
None                58
Has MS              58
No Connection       48
Parent has MS       46
Spouse has MS       44
Other               35
Child has MS        25
Sibling has MS      17
Name: Local Team Captain Primary Connection To MS, dtype: int64

In [17]:
#compute count of values in columns
nationalteams['Local Team Captain Email Domain'].value_counts()

gmail.com                215
yahoo.com                 85
hotmail.com               45
aol.com                   28
comcast.net               19
deloitte.com              19
msn.com                   14
salesforce.com            12
sbcglobal.net             10
kpmg.com                   9
pointb.com                 8
wellsfargo.com             8
att.net                    8
fmr.com                    7
ey.com                     7
nmss.org                   7
teamwingman.org            6
cable.comcast.com          6
us.pwc.com                 5
verizon.net                5
accenture.com              5
biogen.com                 5
live.com                   5
swbell.net                 5
teamaudreysheroes.org      5
me.com                     5
cisco.com                  5
schwab.com                 4
bm.com                     4
pepsico.com                4
                        ... 
monsanto.com               1
samsclub.org               1
samsclub.com               1
bex.net       

In [18]:
#compute count of values in columns
nationalteams['Local Team Captain Contact By Email OK? (Y/N)'].value_counts()

Yes    576
No     174
Name: Local Team Captain Contact By Email OK? (Y/N), dtype: int64

# Describing Data

In [27]:
biketeams.describe()

,Event ID,Team ID,Team Captain Accept Email,Number of Participants,Total Fees Paid,Total Online Gifts($),Total Offline Confirmed Gifts($),Total Offline Unconfirmed Gifts($),Team Goal($),Total Confirmed Gifts in Team History($),Previous Event Fiscal Year,Previous Event Confirmed Gifts($),Fiscal Year
count,33132.000000,33132.000000,33132.0,33132.000000,33132.000000,33132.000000,33132.000000,33132.000000,3.313200e+04,3.313000e+04,8500.000000,3.313200e+04,33130.000000
mean,24480.767777,411722.021248,0.0,10.703429,440.122936,6320.982646,2682.458385,101.641363,6.945165e+04,1.665778e+04,2014.977882,3.677016e+03,2014.938424
std,3081.189125,67696.166584,0.0,21.694745,1512.024337,19529.447014,10539.768813,792.613367,7.769344e+06,6.292724e+04,1.220864,2.087463e+04,1.419012
min,17966.000000,266344.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2011.000000,0.000000e+00,2013.000000
25%,22247.000000,353140.500000,0.0,2.000000,55.000000,405.000000,0.000000,0.000000,0.000000e+00,8.850000e+02,2015.000000,0.000000e+00,2014.000000
50%,25096.000000,405873.500000,0.0,5.000000,140.000000,1600.000000,350.000000,0.000000,2.000000e+03,3.066000e+03,2015.000000,0.000000e+00,2015.000000
75%,27091.000000,466838.250000,0.0,10.000000,340.000000,4920.962500,1571.000000,0.000000,5.000000e+03,1.066500e+04,2016.000000,0.000000e+00,2016.000000
max,29050.000000,538680.000000,0.0,725.000000,71200.000000,751129.190000,498352.720000,39638.000000,1.000000e+09,2.650325e+06,2016.000000,1.249482e+06,2018.000000


In [28]:
# counting 1 values
print((biketeams[['Team Creation Date','Team Captain Contact ID','Captain Email Domain','Team Captain Accept Email','Team Division','Company','Number of Participants','Total Fees Paid','Team Total Confirmed ($)','Total Online Gifts($)','Total Offline Confirmed Gifts($)','Total Offline Unconfirmed Gifts($)','Team Goal($)','Total Confirmed Gifts in Team History($)','Previous Event Fiscal Year','Previous Event Internal Name','Previous Event Team Name','Previous Event Confirmed Gifts($)','Previous Event Team Members','Event Date','Fiscal Year']] == 1).sum())



Team Creation Date                             0
Team Captain Contact ID                        0
Captain Email Domain                           1
Team Captain Accept Email                      0
Team Division                                  0
Company                                        0
Number of Participants                      4515
Total Fees Paid                                1
Team Total Confirmed ($)                       4
Total Online Gifts($)                          2
Total Offline Confirmed Gifts($)               5
Total Offline Unconfirmed Gifts($)             6
Team Goal($)                                   5
Total Confirmed Gifts in Team History($)       4
Previous Event Fiscal Year                     0
Previous Event Internal Name                   0
Previous Event Team Name                       0
Previous Event Confirmed Gifts($)              1
Previous Event Team Members                  534
Event Date                                     0
Fiscal Year         

# FIRST PRIORITY: CORPORATE ACQUISITION
  * What are the greatest growth opportunities for new corporate teams?
  * Can we apply those opportunities to specific rides/markets, especially our biggest events?
  * What industries have had the strongest involvement in Bike MS in the last five years?
  * What occupations were responsible for most of our fundraising?
  * Can we tie together these industries and occupations to identify gaps/opportunities?
  * What is the common denominator for our top performing corporate teams? (Is it industry, culture, executive involvement, connection to MS, other?)
  * Can we quantify the effect competing events are having in our top markets? 
Top competitors:
   1. Best Buddies
     * HYANNIS PORT June 2, 2018 Boston to Hyannis Port, MA
     * HEARST CASTLE September 8, 2018 San Simeon, CA
     * MIAMI November 16, 2018 Miami, FL
   2. Peddle the Cause
     * Louis
     * San Diego
   3. Pelatonia – OH
   4. PanMass Challenge - Boston
   5. ADA – Tour de Cure
   6. Jude Children’s Research Hospital
   7. JDRF Ride to Cure Diabetes
   8. Cycle for Life - Cystic Fibrosis Foundation
   9. Ride to Recovery
   10. Virtual Cycling
   11. Athletic Club Stationary Cycling Relays
   12. American Cancer Society
   13. Leukemia and Lymphoma Society – Team in Training

# SECOND PRIORITY: DIGITAL/SOCIAL ACQUISITION
  * What are the greatest opportunities for digital marketing investments? Where have we seen the greatest ROI?
  * Once someone is registered, what tactics and behaviors drive fundraising, and at what times leading up to the event?
  * What behavioral data do you see about usage of our fundraising tools and how it may or may not relate to performance of top fundraisers?
  * Despite increasing our digital advertising spend, acquisition continues to trend downward overall. Why? Is it an issue of needing more traffic, better targeting, or a conversion rate issue that needs to be addressed through the registration process? What can we do to reverse the trend?


# FINALLY:
  * As you studied this data, is there something else that came up as an insight into our operations that the questions above do not capture? 


# Descriptive Analytics